In [1]:
import os
import sys
import tarfile
import time
import urllib.request


source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)

100% | 80 MB | 1.86 MB/s | 43 sec elapsed

In [2]:
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

In [3]:
!pip install pyprind

In [4]:
import pyprind
import pandas as pd
import os

# `basepath`를 압축 해제된 영화 리뷰 데이터셋이 있는
# 디렉토리로 바꾸세요

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)

data = []

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file),
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            data.append({'review' : txt, 'sentiment' : labels[l]})
            pbar.update()

df = pd.DataFrame(data)
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


In [5]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [6]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [7]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [9]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [10]:
lda.components_.shape

(10, 5000)

In [11]:
n_top_words = 5
feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool




1.   대체적으로 형편없는 영화(실제 토픽 카테고리가 되지 못함)
2.   가족 영화
3.   전쟁 영화
4.   예술 영화
5.   범죄 영화
6.   공포 영화
7.   코미디 영화
8.   TV 쇼와 관련된 영화
9.   소설을 원작으로 한 영화
10.   액션 영화














In [12]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

공포 영화 #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

공포 영화 #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/2024 파이썬 코스 - 1/6. IMDb 영화 데이터 리뷰/olympics-economics.csv', encoding='utf-8')
df.head(3)

,country,country_code,gold,silver,bronze,total,gdp,gdp_year,population
0,United States,USA,40,44,42,126,81695.19,2023,334.9
1,China,CHN,40,27,24,91,12614.06,2023,1410.7
2,Japan,JPN,20,12,13,45,33834.39,2023,124.5


In [42]:
df['description'] = df.apply(lambda row:
                               f"{row['country']} has a GDP of ${row['gdp']:,.2f} with a population of {row['population']} million. "
                               f"This Olympics has a total of {row['total']} medals.",
                               axis=1)

df

,country,country_code,gold,silver,bronze,total,gdp,gdp_year,population,description
0,United States,USA,40,44,42,126,81695.19,2023,334.9,"United States has a GDP of $81,695.19 with a p..."
1,China,CHN,40,27,24,91,12614.06,2023,1410.7,"China has a GDP of $12,614.06 with a populatio..."
2,Japan,JPN,20,12,13,45,33834.39,2023,124.5,"Japan has a GDP of $33,834.39 with a populatio..."
3,Australia,AUS,18,19,16,53,64711.77,2023,26.6,"Australia has a GDP of $64,711.77 with a popul..."
4,France,FRA,16,26,22,64,44460.82,2023,68.2,"France has a GDP of $44,460.82 with a populati..."
...,...,...,...,...,...,...,...,...,...,...
85,Peru,PER,0,0,1,1,7789.87,2023,34.4,"Peru has a GDP of $7,789.87 with a population ..."
86,Qatar,QAT,0,0,1,1,87480.42,2022,2.7,"Qatar has a GDP of $87,480.42 with a populatio..."
87,Singapore,SGP,0,0,1,1,84734.26,2023,5.9,"Singapore has a GDP of $84,734.26 with a popul..."
88,Slovakia,SVK,0,0,1,1,24470.24,2023,5.4,"Slovakia has a GDP of $24,470.24 with a popula..."


In [41]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['gdp'].values)

AttributeError: 'numpy.float64' object has no attribute 'lower'

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

In [ ]:
lda.components_.shape

(10, 5000)

In [ ]:
n_top_words = 5
feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool




1.   대체적으로 형편없는 영화(실제 토픽 카테고리가 되지 못함)
2.   가족 영화
3.   전쟁 영화
4.   예술 영화
5.   범죄 영화
6.   공포 영화
7.   코미디 영화
8.   TV 쇼와 관련된 영화
9.   소설을 원작으로 한 영화
10.   액션 영화














In [ ]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

공포 영화 #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

공포 영화 #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...
